<a href="https://colab.research.google.com/github/sudhanshu2/propagate-alerts/blob/master/simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alert Propagation on EdgeSys
This notebook contains the simulations for the Alert Propagation API implemented on EdgeSys. Please refer to the `README` in the Github repository for further information.

# Graph Creation
This section contains specifications for the graph, including the number of total sensors and the number of sensors each sensor is connected to in the decentralized system. 

In [1]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

def create_current(number_sensors):
  G_curr = nx.Graph()
  for i in range(number_sensors):
    G_curr.add_node(i, visited="false")
    
  for i in range(1, number_sensors):
    G_curr.add_edge(0, i)
  return G_curr

G_curr = create_current(1000)
print("created a graph with " + str(len(G_curr.nodes())) + " nodes and " + str(len(G_curr.edges())) + " edges")

created a graph with 1000 nodes and 999 edges


In [87]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from math import log
import secrets

def create_alert_graph(number_sensors, connected_sensors, random_connections=False):
  random_generator = secrets.SystemRandom()
  max_connected_sensors = connected_sensors

  G_alert = nx.Graph()
  for i in range(number_sensors):
    G_alert.add_node(i, visited="false")
  
  if random_connections is True:
    connected_sensors = random_generator.randrange(1, max_connected_sensors)
  for i in range(1, connected_sensors + 1):
    G_alert.add_edge(0, i)
    
  current_node = 1
  current_difference = connected_sensors
  
  while current_node + current_difference <= number_sensors:
    if random_connections is True:
      connected_sensors = random_generator.randrange(1, max_connected_sensors)
    for i in range(connected_sensors - 1):
      to_join = current_node + current_difference + i
      if current_node < number_sensors and to_join < number_sensors:
        G_alert.add_edge(current_node, to_join)
    current_node += 1
    current_difference += connected_sensors - 2
  return G_alert

G_alert = create_alert_graph(1000, 5)
print("created a graph with " + str(len(G_alert.nodes())) + " nodes and " + str(len(G_alert.edges())) + " edges")

created a graph with 1000 nodes and 999 edges


# Graph Visualization
This section contains visualizations for the graphs.

In [89]:
import networkx as nx

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx

output_notebook()

G = G_curr

plot = figure(title="Centralized Node Network", x_range=(-2.2,2.2), y_range=(-2.2,2.2),
              tools="", toolbar_location=None)

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.axis.visible = False
plot.renderers.append(graph)

show(plot)

In [90]:
import networkx as nx

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx

output_notebook()

G = G_alert

plot = figure(title="Edge Based Node Network", x_range=(-2.2,2.2), y_range=(-2.2,2.2),
              tools="", toolbar_location=None)

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.axis.visible = False
plot.renderers.append(graph)

show(plot)

# Simulation
This section contains the code for simulating the network structure under different conditions. The ```G``` specied in the first code block is the graph used for simulations.  



In [64]:
from collections import defaultdict

def alert_time_simulation(G, alert_origin, multiplier=1.0):
  for node in G.nodes:
    G.nodes[node]["visited"] = "false"

  normalized_latency_multiplier = (200.0 / len(G.nodes())) * multiplier
    
  not_visited_neighbors = list()
  G_visited = defaultdict(list)
    
  not_visited_neighbors.append(alert_origin)
  G.nodes[alert_origin]["visited"] = "true"
  G_visited[0].append(alert_origin)  
  time_stamps = set()
    
  while len(not_visited_neighbors) != 0:
    current = not_visited_neighbors.pop()
    for edge in G.edges(current):
      if G.nodes[edge[1]]["visited"] == "false":
        not_visited_neighbors.append(edge[1])
        G.nodes[edge[1]]["visited"] = "true"
        latency = int(normalized_latency_multiplier * abs(edge[1] - edge[0]))
        G_visited[latency].append(edge[1])      
        time_stamps.add(latency)
          
  sorted_time_stamps = list(time_stamps)
  sorted_time_stamps.sort()
  
  time_stamp = list()
  nodes_alerted = list()
  current_nodes_alerted = 0
  
  for i in range(sorted_time_stamps[len(sorted_time_stamps) - 1] + 1):
    if i in G_visited:
      current_nodes_alerted += len(G_visited[i])
    time_stamp.append(i)
    nodes_alerted.append(current_nodes_alerted)
  return time_stamp, nodes_alerted

### Simulation Decentralized Alert 
This is a set of simulations exclusive to the decentralized alert system.

#### Different Number of Nodes

In [84]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row
import secrets

MIN_NODES = 100
MAX_NODES = 1000
NODE_CONNECTIONS = 10
STEP = 10

current_node_count = MIN_NODES

xs = list()
ys = list()
xs_random = list()

while current_node_count <= MAX_NODES:
  G_alert = create_alert_graph(current_node_count, NODE_CONNECTIONS)
  G_alert_random = create_alert_graph(current_node_count, NODE_CONNECTIONS, True)

  random_generator = secrets.SystemRandom()
  alert_origin = random_generator.randrange(1, current_node_count - 1)

  time_stamp, nodes_alerted = alert_time_simulation(G_alert, alert_origin)
  time_stamp_random, nodes_alerted_random = alert_time_simulation(G_alert_random, alert_origin)

  xs.append(len(time_stamp))
  xs_random.append(len(time_stamp_random))
  ys.append(current_node_count)

  current_node_count += STEP

output_notebook()

p_fixed = figure(title="Fixed number of connections per node", plot_width=400, plot_height=400)
p_fixed.circle(xs, ys, size=5, color="#000000", alpha=0.5)
p_fixed.xaxis.axis_label = "total time"
p_fixed.yaxis.axis_label = "total nodes"
p_fixed.toolbar.logo = None
p_fixed.toolbar_location = None

p_random = figure(title="Random number of connections per node", plot_width=400, plot_height=400)
p_random.circle(xs_random, ys, size=5, color="#000000", alpha=0.5)
p_random.xaxis.axis_label = "total time"
p_random.yaxis.axis_label = "total nodes"
p_random.toolbar.logo = None
p_random.toolbar_location = None

p = row(p_fixed, p_random)
show(p)

In [85]:
from bokeh.plotting import figure, show, output_notebook
import secrets

MIN_NODES = 100
MAX_NODES = 1000
NODE_CONNECTIONS = 10
STEP = 10

current_node_count = MIN_NODES

xs = list()
ys = list()
xs_random = list()
ys_random = list()

while current_node_count <= MAX_NODES:
  G_alert = create_alert_graph(current_node_count, NODE_CONNECTIONS)
  G_alert_random = create_alert_graph(current_node_count, NODE_CONNECTIONS, True)

  random_generator = secrets.SystemRandom()
  alert_origin = random_generator.randrange(1, current_node_count - 1)

  time_stamp, nodes_alerted = alert_time_simulation(G_alert, alert_origin)
  time_stamp_random, nodes_alerted_random = alert_time_simulation(G_alert_random, alert_origin)

  xs.append(time_stamp)
  ys.append(nodes_alerted)

  xs_random.append(time_stamp_random)
  ys_random.append(nodes_alerted_random)

  current_node_count += STEP

output_notebook()

p_fixed = figure(title="Fixed number of connections per node", plot_width=400, plot_height=400)
p_fixed.xaxis.axis_label = "time"
p_fixed.yaxis.axis_label = "nodes"
r_fixed = p_fixed.multi_line(xs, ys, color="#000000", line_alpha=0.6, line_width=0.5)
p_fixed.add_layout(r_fixed)
p_fixed.toolbar.logo = None
p_fixed.toolbar_location = None

p_random = figure(title="Random number of connections per node", plot_width=400, plot_height=400)
p_random.xaxis.axis_label = "time"
p_random.yaxis.axis_label = "nodes"
r_random = p_random.multi_line(xs_random, ys_random, color="#000000", line_alpha=0.6, line_width=0.5)
p_random.add_layout(r_random)
p_random.toolbar.logo = None
p_random.toolbar_location = None

p = row(p_fixed, p_random)
show(p)

#### Different Origins

In [93]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row
import secrets

NODES = 1000
NODE_CONNECTIONS = 10
NUMBER_LOOPS = 100

count = 0

xs = list()
ys = list()
xs_random = list()
ys_random = list()

while count <= NUMBER_LOOPS:
  G_alert = create_alert_graph(NODES, NODE_CONNECTIONS)
  G_alert_random = create_alert_graph(NODES, NODE_CONNECTIONS, True)

  random_generator = secrets.SystemRandom()
  alert_origin = random_generator.randrange(1, NODES - 1)

  time_stamp, nodes_alerted = alert_time_simulation(G_alert, alert_origin)
  time_stamp_random, nodes_alerted_random = alert_time_simulation(G_alert_random, alert_origin)

  xs.append(len(time_stamp))
  xs_random.append(len(time_stamp_random))
  ys.append(alert_origin)

  count += 1

output_notebook()

p_fixed = figure(title="Fixed number of connections per node", plot_width=400, plot_height=400)
p_fixed.circle(xs, ys, size=5, color="#000000", alpha=0.5)
p_fixed.xaxis.axis_label = "total time"
p_fixed.yaxis.axis_label = "alert origin node"
p_fixed.toolbar.logo = None
p_fixed.toolbar_location = None

p_random = figure(title="Random number of connections per node", plot_width=400, plot_height=400)
p_random.circle(xs_random, ys, size=5, color="#000000", alpha=0.5)
p_random.xaxis.axis_label = "total time"
p_random.yaxis.axis_label = "alert origin node"
p_random.toolbar.logo = None
p_random.toolbar_location = None

p = row(p_fixed, p_random)
show(p)

In [97]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row
import secrets

NODES = 1000
NODE_CONNECTIONS = 100
NUMBER_LOOPS = 100
STEP = 1

current_connections = 5

xs = list()
ys = list()
xs_random = list()
ys_random = list()

while current_connections <= NODE_CONNECTIONS:
  G_alert = create_alert_graph(NODES, current_connections)
  G_alert_random = create_alert_graph(NODES, current_connections, True)

  random_generator = secrets.SystemRandom()
  alert_origin = random_generator.randrange(1, NODES - 1)

  time_stamp, nodes_alerted = alert_time_simulation(G_alert, alert_origin)
  time_stamp_random, nodes_alerted_random = alert_time_simulation(G_alert_random, alert_origin)

  xs.append(len(time_stamp))
  xs_random.append(len(time_stamp_random))
  ys.append(current_connections)

  current_connections += STEP

output_notebook()

p_fixed = figure(title="Fixed number of connections per node", plot_width=400, plot_height=400)
p_fixed.circle(xs, ys, size=5, color="#000000", alpha=0.5)
p_fixed.xaxis.axis_label = "total time"
p_fixed.yaxis.axis_label = "connections per node"
p_fixed.toolbar.logo = None
p_fixed.toolbar_location = None

p_random = figure(title="Random number of connections per node", plot_width=400, plot_height=400)
p_random.circle(xs_random, ys, size=5, color="#000000", alpha=0.5)
p_random.xaxis.axis_label = "total time"
p_random.yaxis.axis_label = "maximum connections per node"
p_random.toolbar.logo = None
p_random.toolbar_location = None

p = row(p_fixed, p_random)
show(p)

### Alert Time Simulation

In [178]:
time_stamp, nodes_alerted = alert_time_simulation(G_curr, 0.6)
print(time_stamp)
print(nodes_alerted)

normalized latency for edges: 0.12
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
[9, 17, 25, 34, 42, 50, 59, 67, 75, 84, 92, 100, 109, 117, 125, 134, 142, 150, 159, 167, 175, 184, 192, 200, 209, 217, 225, 234, 242, 250, 259, 267, 275, 284, 292, 300, 309, 317, 325, 334, 342, 350, 359, 367, 375, 384, 392, 400, 409, 417, 425, 434, 442, 450, 459, 467, 475, 484, 492, 500, 509, 517, 525, 534, 542, 550, 559, 567, 575, 584, 592, 600, 609, 617, 625, 634, 642, 650, 659, 667, 675, 684, 692, 700, 709, 717, 725, 734, 742, 750, 759, 767, 775, 784, 792, 800, 809, 

In [179]:
time_stamp, nodes_alerted = alert_time_simulation(G_alert, )
print(time_stamp)
print(nodes_alerted)

normalized latency for edges: 0.2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
[5, 12, 19, 25, 32, 39, 45, 52, 59, 65, 72, 79, 85, 92, 99, 105, 112, 119, 125, 132, 139, 145, 152, 159, 165, 172, 179, 185, 192, 199, 205, 212, 219, 225, 232, 239, 245, 252, 259, 265, 272, 279, 285, 292, 299, 305, 312, 319, 325, 332, 339, 345, 352, 359, 365, 372, 379, 385, 392, 399, 405, 412, 419, 425, 432, 439, 445, 